# Intro to Embeddings

In this notebook, we learn on how to build good embeddings for words, through sequential data (sentences) - in other words, "Word2Vec". We are not so much interested in the NLP part of it, but in a more general point: if we pick a good prediction task (such as guessing the next word in a sentence), we will obtain "good embeddings", that is embeddings that are good representations of the underlying concepts.

You are encouraged to play around with the code and modify / re-built parts of it as you fit: there is NO substitute for "tinkering with code" to understand how all the concepts fit together (corollary: all this code is written for pedagogical purposes, so some functions are re-used from previous lectures to provide a self-sufficient script).

In [ ]:
# some global imports
import json
import glob
import os
import pandas as pd
from collections import Counter
from random import sample
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline

## Data loading

In [ ]:
# make sure you have the datasets library installed
# see: https://github.com/huggingface/datasets

# !pip install datasets

In [ ]:
import string

# some utils function
def get_finance_sentiment_dataset(split: str='sentences_allagree'):
    # load financial dataset from HF
    from datasets import load_dataset
    # https://huggingface.co/datasets/financial_phrasebank
    # by default, load just sentences for which all annotators agree
    dataset = load_dataset("financial_phrasebank", split)
    
    return dataset['train']


def get_finance_sentences():
    dataset = get_finance_sentiment_dataset()
    cleaned_dataset = [[pre_process_sentence(_['sentence']), _['label']] for _ in dataset]
    # debug 
    print("{} cleaned sentences from finance dataset\n".format(len(cleaned_dataset)))
    
    return cleaned_dataset


def pre_process_sentence(sentence: str):
    # this choices are VERY important. Here, we take a simplified 
    # view, remove the punctuations and just lower case everything
    lower_sentence = sentence.lower()
    exclude = set(string.punctuation)
    return ''.join(ch for ch in lower_sentence if ch not in exclude)

In [ ]:
finance_dataset = get_finance_sentences()
# print out the first items in the dataset, to check the format
finance_dataset[:2]

In [ ]:
# get sentences without label for vectorizer part
finance_dataset_sentences = [_[0] for _ in finance_dataset]

## From words to embeddings with Word2Vec

Let us use word2vec to get vectors for words first, and document after. We will use a fantastic Python library, gensim: https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
#!pip install gensim==4.2

In [ ]:
import gensim

In [ ]:
def train_word2vec_model(
    sentences: list,
    min_count: int = 2,
    vector_size: int = 48,
    window: int = 2,
    epochs: int = 20
):
    """
    Sentences is a list of lists, where each list is composed by tokens in a sentence: e.g.
    
    [
        ['the', 'cat', 'is', 'on' ...],
        ['i', 'live', 'in', 'nyc', ...],
        ....
    ]
    
    """
    model =  gensim.models.Word2Vec(sentences=sentences,
                                    min_count=min_count,
                                    vector_size=vector_size,
                                    window=window,
                                    epochs=epochs)
    
    # this is how many words we will have in the space
    print("# words in the space: {}".format(len(model.wv.index_to_key)))

    # we return the space in a format that will allow us to do nice things afterwards ;-)    
    return model.wv

In [ ]:
# let's use nltk tokenizer to break up sentences and build a word2vec model
# https://www.nltk.org/api/nltk.tokenize.html
from nltk.tokenize import word_tokenize

print(finance_dataset_sentences[0], '\n\n', word_tokenize(finance_dataset_sentences[0]))

In [ ]:
tokenized_sentences = [word_tokenize(_) for _ in finance_dataset_sentences]
# debug 
tokenized_sentences[0][:5]

In [ ]:
# build a counter to get a sense of the lexicon
word_counter = Counter([item for sent in tokenized_sentences for item in sent])
word_counter.most_common(10)

In [ ]:
w2v_model = train_word2vec_model(tokenized_sentences)

In [ ]:
# each word now has one vector corresponding to it ->
w2v_model['the']

Now that we have a vector space, let's find words similar to a given term...

In [ ]:
for w in ['company', 'profit']:
    print('\n======>{}\n'.format(w), w2v_model.similar_by_word(w, topn=3))

To get a sense of what the vectors look like, we print them out in 2D using TSNE (https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html):

In [ ]:
def plot_scatter_by_category_with_lookup(title, 
                                         words, 
                                         word_to_target_cat,
                                         results):
    """
    Just a plotting routine
    """
    
    groups = {}
    for word, target_cat in word_to_target_cat.items():
        if word not in words:
            continue

        word_idx = words.index(word)
        x = results[word_idx][0]
        y = results[word_idx][1]
        if target_cat in groups:
            groups[target_cat]['x'].append(x)
            groups[target_cat]['y'].append(y)
        else:
            groups[target_cat] = {
                'x': [x], 'y': [y]
                }
    
    fig, ax = plt.subplots(figsize=(10, 10))
    for group, data in groups.items():
        ax.scatter(data['x'], data['y'], 
                   alpha=0.1 if group == 0 else 0.8, 
                   edgecolors='none', 
                   s=25, 
                   marker='o',
                   label=group)

    plt.title(title)
    plt.legend(loc=2)
    plt.show()
    
    return

In [ ]:
def tsne_analysis(embeddings, perplexity=25, n_iter=1000):
    tsne = TSNE(n_components=2, verbose=1, perplexity=perplexity, n_iter=n_iter)
    return tsne.fit_transform(embeddings)

In [ ]:
# map words to known categories of interest

# 0 is the generic category
words = w2v_model.index_to_key
print(len(words))
words_to_category = {w: 0 for w in words}
# manually pick some words to display
for w in ['company', 'profit', 'investment', 'loss', 'margin', 'group']:
    words_to_category[w] = 1
for w in ['with', 'of', 'from', 'by', 'as']:
    words_to_category[w] = 2

In [ ]:
embeddings = [w2v_model[w] for w in words]
tsne_results = tsne_analysis(embeddings)
assert len(tsne_results) == len(words)

In [ ]:
plot_scatter_by_category_with_lookup('Finance trained word2vec', words, words_to_category, tsne_results)

_Why the quality is not ideal?_

Our dataset is very small, and word2vec works much better when large corpora are used. What if we trained the vectors on a much bigger dataset, say, Wikipedia?

We don't have to, as we can donwload embeddings for words as gently pretrained by other researchers - let's do the same qualitative checks, but on Wikipedia embeddings!

In [ ]:
import gensim.downloader as api

In [ ]:
# glove-wiki-gigaword-50 (400000 records): Pre-trained vectors based on Wikipedia
pre_trained_model = api.load("glove-wiki-gigaword-50")
# test it out
for w in ['company', 'profit']:
    print('\n======>{}\n'.format(w), pre_trained_model.similar_by_word(w, topn=3))

In [ ]:
words =[w for w in w2v_model.index_to_key if w in pre_trained_model]
print(len(words))
pre_trained_vectors = [pre_trained_model[w] for w in words]
pre_trained_tsne_results = tsne_analysis(pre_trained_vectors)

In [ ]:
plot_scatter_by_category_with_lookup('Wikipedia trained word2vec', 
                                     words, 
                                     words_to_category, 
                                     pre_trained_tsne_results)

_Let's try some other categories on the full Wikipedia corpus!_

In [ ]:
# build a new mapping and manually pick some words to display
wiki_words_to_category = {}
for w in ['dog', 'cat', 'bunny', 'horse', 'snake']:
    wiki_words_to_category[w] = 1
for w in ['with', 'of', 'from', 'by', 'as']:
    wiki_words_to_category[w] = 2
for w in ['italy', 'france', 'spain', 'germany', 'portugal', 'china']:
    wiki_words_to_category[w] = 3
for w in ['rome', 'berlin', 'paris', 'milan', 'madrid']:
    wiki_words_to_category[w] = 4
for w in ['tennis', 'soccer', 'basketball', 'volleyball', 'football']:
    wiki_words_to_category[w] = 5
for w in ['dress', 'skirt', 'pants', 'trousers', 'shirt', 'denim']:
    wiki_words_to_category[w] = 6
for w in ['pasta', 'pizza', 'dumplings', 'ramen', 'sushi', 'burger']:
    wiki_words_to_category[w] = 7
for w in ['dollar', 'yen', 'euro', 'rupee']:
    wiki_words_to_category[w] = 8
    
# limit to 10000 words to speed up TSNE    
wiki_words = list(set(sample(pre_trained_model.index_to_key, 1000) + list(wiki_words_to_category.keys()))) 
for w in wiki_words:
    if w not in wiki_words_to_category:
        wiki_words_to_category[w] = 0

# display
wiki_pre_trained_vectors = [pre_trained_model[w] for w in wiki_words]
wiki_pre_trained_tsne_results = tsne_analysis(wiki_pre_trained_vectors)
plot_scatter_by_category_with_lookup('Wikipedia trained word2vec', 
                                     wiki_words, 
                                     wiki_words_to_category, 
                                     wiki_pre_trained_tsne_results)

### Bonus: word2vec for analogies

A famous property of word2vec is the ability to capture analogical relations through the embedding space, such as for example:

man : king = woman : ?

We re-use a pre-trained model, trained on wikipedia, to show how analogies are encoded in the latent space.

In [ ]:
# get a model trained on twitter: https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html
pre_trained_model = api.load("glove-twitter-50")

_Note that most_similar is performing here a KNN on the vector space using cosine similarity_ 

In [ ]:
def analogy(model, worda, wordb, wordc):
    result = model.most_similar(negative=[worda], 
                                positive=[wordb, wordc])
    return result[0][0]

In [ ]:
print(analogy(pre_trained_model, 'king', 'man', 'queen'))

In [ ]:
countries = ['australia', 'canada', 'germany', 'ireland', 'italy']
foods = [analogy(pre_trained_model, 'us', 'hamburger', country) for country in countries]

In [ ]:
for c, f in zip(countries, foods):
    print(c, f)